In [30]:
from bs4 import BeautifulSoup as bs
import requests
from tqdm import tqdm 
import json
import os
import datetime
import re
import csv


### 1. Data collection
## 1.1. Get the list of places

In [2]:
total_link = []
s = requests.Session()
for n in tqdm(range(1,401)):
    url = f'https://www.atlasobscura.com/places?page={n}&sort=likes_count'
    result = s.get(url)
    soup = bs(result.text)
    puf = soup.find_all("a", {'class': 'content-card content-card-place'})
    for x in puf:
        total_link.append(x['href'])


100%|██████████| 400/400 [16:59<00:00,  2.55s/it]


In [13]:
print(len(total_link))

7200


## 1.2. Crawl places

In [5]:
def downloadPage(start,end, array):
    count_link = ((start-1)*18)+1
    count_page = start
    
    parent_dir = f'C:/Users/auror/Desktop/data_science/ADM/HM/3/html-20221104T081036Z-001/html/'
    s = requests.Session()

    for x in tqdm(range(start, end)):
        if count_link%30 == 0:
            s = requests.Session()
        if count_link%18 == 1:
            #directory = f'folder_{count_page}'
            path = os.path.join(parent_dir, f"folder_{count_page}")
            os.mkdir(path)
        for y in range(18):
            url = f'https://www.atlasobscura.com{array[count_link-1]}'
            name_file = f'location_{count_link}'
            name_folder = f'folder_{count_page}'
            with open(f'./{name_folder}/{name_file}.html', 'w', encoding='utf8') as fp:
                fp.write(s.get(url).text)
            if count_link%18 == 0:
                count_page += 1
            count_link += 1
           
        

In [4]:
downloadPage(133, 168, total_link)

100%|██████████| 35/35 [32:16<00:00, 55.32s/it]


## 1.3 Parse downloaded pages

In [11]:
os.mkdir("TSV Files")
for x in os.listdir():
    for y in os.listdir(x):
        with open(f'{x}/{y}', encoding='utf8') as f:
            p = f.read()
            soup =  bs(p)
            pageAttribute = []
            placeName = findPlaceName(soup)
            placeTags = findPlaceTags(soup)
            numPeopleVisited = findNumPeopleVisited(soup)
            numPeopleWant = findNumPeopleWant(soup)
            placeDesc = findDescription(soup)
            placeShortDesc = findShortDescription(soup)
            placeNearby = findNearbyPlaces(soup)
            placeAddress = findAddress(soup)
            placeAlt, placeLong = findCordinates(soup)
            placeEditors = findPostEditors(soup)
            placePubDate = findPublishingDate(soup)
            placeRelatedList = findPlaceRelatedList(soup)
            placeRelatedPlaces = findRelatedPlaces(soup)
            placeURL = findPageURL(soup)
            with open(f'./TSV Files/{y[:-5]}.tvs', 'wt', encoding='utf8') as fp:
                tsv_writer = csv.writer(fp, delimiter='\t')
                tsv_writer.writerow([placeName, placeTags, numPeopleVisited, numPeopleWant, placeDesc, placeShortDesc, placeNearby, placeAddress, placeAlt, placeLong, placeEditors, placePubDate, placeRelatedList, placeRelatedPlaces, placeURL])

IndexError: list index out of range

In [29]:
def findPlaceName(soup):
    placeName = soup.find("h1", {"class": "DDPage__header-title"}).text
    #placeName = re.sub('[A-Za-z0-9_.,! "]*' ,'',placeName)
    return placeName

def findPlaceTags(soup):
    tags = []
    placeTags = soup.find_all("a", {"class": "itemTags__link js-item-tags-link"})
    for tag in placeTags:
        t = tag.text.replace("\n", "")
        #t = re.sub('[A-Za-z0-9 _.,!"]*','',t)
        tags.append(t)
    return tags

def findNumPeopleVisited(soup):
    return int(soup.find_all("div", {"class": "title-md item-action-count"})[0].text)

def findNumPeopleWant(soup):
    return int(soup.find_all("div", {"class": "title-md item-action-count"})[1].text)

def findDescription(soup):
    all_description = []
    descriptions = soup.find_all("div", {"class": "DDP__body-copy"})
    for description in descriptions:
        d = description.text.replace("\n","")
        #d = re.sub('[A-Za-z0-9 _.,!"]*','',d)
        all_description.append(d)
    return all_description

def findShortDescription(soup):
    shortDescription = soup.find("h3", {"class": "DDPage__header-dek"}).text.replace("\n", "")
    #shortDescription = re.sub('[A-Za-z0-9 _.,!"]*','',shortDescription)
    return shortDescription

def findNearbyPlaces(soup):
    nearPlaces = []
    nearbyPlaces = soup.find_all("div", {"class": "DDPageSiderailRecirc__item-title"})
    for place in nearbyPlaces:
        p = place.text.replace("\n","")
        #p = re.sub('[A-Za-z0-9 _.,!"]*','',p)
        nearPlaces.append(p)
    #Convert the list to set, and then back again to list to remove repetition
    return list(set(nearPlaces))

def findAddress(soup):
    strings = []
    adress_strings = soup.find("div", {"class":"DDPageSiderail__column grid-col-lg-4 grid-col-md-5"}).find("aside", {"class": "DDPageSiderail__details"}).find("address", {"class": "DDPageSiderail__address"}).find("div")
    for info in adress_strings:
        s = info.text.replace("\n", "")
        if s != "":
            strings.append(s)
    return " ".join(strings[:3])

def findCordinates(soup):
    cordinates = soup.find("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"})
    placeAlt, placeLong = cordinates.text.replace("\n", "").replace(" ","").split(",")
    return placeAlt, placeLong

def findPostEditors(soup):
    all_editors =[]
    editors = soup.find_all("a", {"class":"DDPContributorsList__contributor"})         
    for person in editors:
        s = person.text.replace("\n", "")
        all_editors.append(s)
    return all_editors

def findPublishingDate(soup):
    #Pick the right info
    dateString = soup.find("div", {"class":"DDPContributor__name"}) 
    #Let's clean the string
    s = dateString.text.replace("\n", "")
    #Let's modify it for the right format of datetime
    split = s.split()
    #Let's convert the string Month into the corrispondent number by using "strptime()" 
    split[0] = str(datetime.datetime.strptime(split[0], '%B').month)
    #My format
    format = "%m %d, %Y"
    #Convert from String to datetime
    date = datetime.datetime.strptime(" ".join(split), format)
    return date

def findPlaceNear(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Nearby"}).find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
    for list in relatedLists:
        s = list.text.replace("\n", "")
        #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
        lists.append(s)
    return lists

def findRelatedPlaces(soup):
    lists =[]
    relatedLists = soup.find("div", {"data-gtm-template":"DDP Footer Recirc Related"}).find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
    for list in relatedLists:
        s = list.text.replace("\n", "")
        #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
        lists.append(s)
    return lists

# def findRelatedPlaces(soup):
#     relatedPlaces = []
#     if soup.find_all("div", {"class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})[1]!=None:
#         places = soup.find_all("div", {"class":"card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})[1].find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})
#         for relatedPlace in places:
#             s = relatedPlace.text.replace("\n", "")
#             #s = re.sub('[A-Za-z0-9 _.,!"]*','',s)
#             relatedPlaces.append(s)
#     return relatedPlaces

def findPageURL(soup):
    numVisitedPeople = soup.find("link", {"rel":"canonical"})
    return numVisitedPeople['href']



In [36]:
for x in os.listdir():
    for y in os.listdir(x):
        with open(f'{x}/{y}', encoding='utf8') as f:
                    p = f.read()
                    soup =  bs(p)
                    near = findPlaceNear(soup)
                    related = findRelatedPlaces(soup)
                    print(related)


        break
    break


['Crystal Palace Subway', 'Moscow Metro Stations', 'Rove Tunnel', 'Klafthmonos Square Parking Structure Fortification Walls']
